In [1]:
'''from google.colab import drive
drive.mount('/content/drive')'''

Mounted at /content/drive


In [2]:
! pip install transformers[torch] datasets rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-1

In [3]:
!pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 20.5 MB/s eta 0:00:00


In [4]:
import warnings
warnings.filterwarnings("ignore")

### Load Dataset ###

In [5]:
from datasets import load_dataset
dataset = load_dataset("Samsung/samsum")

The repository for Samsung/samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Samsung/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [6]:
dataset.column_names

{'train': ['id', 'dialogue', 'summary'],
 'test': ['id', 'dialogue', 'summary'],
 'validation': ['id', 'dialogue', 'summary']}

In [7]:
max_input = 512
max_target = 128
batch_size = 3

In [8]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def preprocess_data(data_to_process):
  inputs = ["summarize:"+ str(dialogue) for dialogue in data_to_process['dialogue']]

  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)

  model_inputs['labels'] = targets['input_ids']

  return model_inputs

In [10]:
tokenize_data = dataset.map(preprocess_data, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [11]:
tokenize_data.items()

dict_items([('train', Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})), ('test', Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})), ('validation', Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 818
}))])

In [16]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

arg = Seq2SeqTrainingArguments(output_dir = '/content/drive/MyDrive/test_run_0.1',
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 1,
    predict_with_generate = True,
    fp16 = True)

In [20]:
from datasets import load_metric
import numpy as np
import nltk
nltk.download('punkt')

metric = load_metric('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}


    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [21]:
trainer = Seq2SeqTrainer(
    model,
    arg,
    train_dataset=tokenize_data["train"],
    eval_dataset=tokenize_data["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.452700,0.407310,39.847300,17.209900,33.403400,36.939400,16.328900


TrainOutput(global_step=4911, training_loss=0.4473601014811695, metrics={'train_runtime': 761.1484, 'train_samples_per_second': 19.355, 'train_steps_per_second': 6.452, 'total_flos': 1993855419285504.0, 'train_loss': 0.4473601014811695, 'epoch': 1.0})

In [23]:
trainer.evaluate()

{'eval_loss': 0.40731024742126465,
 'eval_rouge1': 39.8473,
 'eval_rouge2': 17.2099,
 'eval_rougeL': 33.4034,
 'eval_rougeLsum': 36.9394,
 'eval_gen_len': 16.3289,
 'eval_runtime': 106.5957,
 'eval_samples_per_second': 7.674,
 'eval_steps_per_second': 2.561,
 'epoch': 1.0}

In [28]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the trained model and tokenizer
model_name = "/content/drive/MyDrive/test_run_0.1/checkpoint-4500"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare the test case
input_text = '''A Facebook Live video posted Saturday by the sheriff's office showed Collins and Johnson being escorted out of a residence and placed in a police vehicle.

Chief Deputy Christy Sykes said Collins and Johnson climbed through a panel in the ceiling in a common area of the jail. They made their way to a window and used bed sheets to drop down to the ground, she said. Jail staff were alerted to the escape immediately by other inmates.

Johnson injured his leg during the escape and had a bad limp, authorities said in an earlier Facebook post.

Dezarrious Johnson
Dezarrious Johnson.Claiborne County Sheriff's Dept.
Sykes said Collins and Johnson did not exit the facility with any weapons and authorities did not believe they were armed, but had cautioned that the pair should be considered dangerous.

The sheriff's office had urged residents to "take necessary precautions."

The Mississippi Bureau of Investigation had said the two men should not be approached.

Collins was being held for charges out of Copiah County, the sheriff's office said. According to NBC affiliate WLBT of Jackson Collins was arrested in 2023 in connection with the killing of his 46-year-old cousin.

Johnson was being held for charges out of Jefferson County.

Sykes said Friday that Collins and Johnson faced additional charges once they were captured.

Sykes said this isn't the first time inmates at the facility were able to escape through a panel in the ceiling.

"It's been an issue," she said. "We're working closely with the board of supervisors to get this taken care of as soon as possible."'''

# Tokenize the input text
inputs = tokenizer(input_text, max_length=512, truncation=True, return_tensors="pt")

# Move inputs to the model's device (GPU if available)
input_ids = inputs.input_ids.to(model.device)
attention_mask = inputs.attention_mask.to(model.device)

# Generate the summary
outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=128, num_beams=4)

# Decode the generated summary
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Summary:", generated_summary)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generated Summary: . Collins and Johnson climbed through a panel in the ceiling in a common area of the jail. They made their way to a window and used bed sheets to drop down to the ground. They did not exit the facility with any weapons and authorities did not believe they were armed.
